# Exploitation statistique de la base de données morphologique Démonette

L'objectif de ce tutoriel est de présenter un exemple d'exploitation statistique de la base de données morphologiques Démonette produite dans le cadre du projet Demonext (ANR-17-CE23-0005, https://www.demonext.xyz/en/home/). Le tutoriel se base sur l'exemple de l'analyse de la concurrence affixale pour la formation de gentilés en français, et reprend de façon simplifiée le travail de Thuilier et al. (soumis).

Le tutoriel propose les bases de la description de données sous R. Il n'intègre pas de modélisation statistique avancée puisqu'il n'y a pas assez de données pour que cela ait un sens statistiquement. Nous renvoyons notamment vers Thuilier et al. (soumis) pour un exemple plus complet intégrant des traitements statistiques plus poussés.

Le tutoriel présente les étapes suivantes :
- Extraction des données depuis Démonette ;
- Chargement des données sous R ;
- Exploration des données ;
- Manipulation des données ;
- Description des données ;
- Évaluation statistique ;


## Extraction des données depuis Démonette 

Ce travail repose sur l'exploitation des données contenues dans la ressource Démonette, interrogeable par le biais de deux interfaces : 
- la table des lexèmes (http://demonext.llf-paris.fr/demonext/vues/table_lexemes.php) ;
- la table des relations (http://demonext.llf-paris.fr/demonext/vues/table_relations.php). 

La première permet d'explorer les lexèmes sur la base de leurs propriétés individuelles (forme phonologique ou graphique, catégorie, etc), alors que la seconde permet d'intégrer des informations liées à la construction du lexème et à sa famille morphologique (ascendante ou descendante). C'est à partir de cette interface que nous allons travailler puisque dans ce tutoriel, nous nous intéressons aux gentilés construits en -*ais*, -*éen*, -*ien* et -*ois* sur la base de toponymes. 

Nous présentons ici les étapes de sélection à réaliser directement sur l'interface (http://demonext.llf-paris.fr/demonext/vues/table_relations.php). L'importation des données dans le présent tutoriel sera faite dans des étapes ultérieures.

Notre objectif est de récupérer des paires gentilé-toponyme. Pour cela, les colonnes qui nous intéressent (et que nous exporterons) sont donc **graph_1** qui va désigner ici le lexème dérivé (le candidat gentilé) et **graph_2** qui désignera ici le lexème base (le candidat toponyme). Dans les autres colonnes (par ex. **cat_2**, **constr_1**, etc.), 1 réfèrera donc toujours au lexème dérivé et 2 au lexème base. Les critères à prendre en compte pour la sélection des données sont:
- le fait que le lexème base soit un toponyme
- la présence de l'un des quatre suffixes ciblés sur le lexème dérivé

Il n'est pas encore possible dans Demonette de savoir si un lexème est un toponyme. En revanche, nous pouvons nous baser sur les catégories pour filtrer les lexèmes bases afin de ne récupérer que ceux correspondant à un nom propre. Pour cela, nous indiquons "Np" (pour "nom propre") dans le champ **cat_2** (pour la catégorie du lexème 2, c'est-à-dire le lexème base). Dans Demonette, les noms propres sont codés "Npf" pour les noms propres féminins (par ex. *Catherine*), "Npfp" pour les noms propres féminins pluriel (par ex. *Flandres*), "Npm" pour les noms propres masculins (par ex. *Charles*) et "Npx" pour les noms propres dont le genre n'est pas spécifié (par ex. *Goth*). Taper "Np" dans le champ **cat_2** permet de récupérer tous les noms propres, qu'ils soient féminins, masculins ou non spécifiés pour le genre.

Pour identifier les lexèmes dérivés en nous basant sur la présence des suffixes recherchés il existe plusieurs méthodes équivalentes. Nous n'en présentons qu'une seule. Quelle que soit la méthode, il est nécessaire de cocher l'option "Recherche avec des REGEXP", en haut à droite sur l'interface, qui permet de faire des recherches plus poussées grâce aux expressions régulières. Nous utilisons uniquement le champ **cstr_1** qui indique la construction du lexème dérivé, et nous faisons 4 requêtes différentes, chacune correspondant aux différents suffixes, afin de garder la trace du suffixe étudié. Pour les suffixés en -*ais*, nous avons donc saisi "\^Xais\$" dans le champ **cstr_1**. "Xais" indique qu'il doit y avoir la chaîne de caractères *ais* après la base, représentée par X. "$" permet de préciser que la chaîne de caractères *ais* soit se trouver en fin de mot. Cette précision est nécessaire, sans quoi nous récupérons les constructions *Xaisien* (par exemple *salaisien* construit sur *Salles*) ou *Xaisisme* (par exemple *lyonnaisisme* construit sur *Lyon*). "\^" au début du patron de construction est églament important car cela permet d'indiquer que X consitue le début de la construction. Cette précision est elle aussi nécessaire, sans quoi nous récupérons *antifrançais*, dont la construction est "antiXais". Il est important de noter que nous n'avons pas mis de condition sur la catégorie du lexème dérivé, qui peut être adjectif ou nom. Dans plusieurs cas, à la fois le nom et l'adjectif sont indiqués dans Demonette. Dans une étape ultérieure les doublons ont été supprimés car la catégorie du gentilé ne nous intéresse pas pour cette étude.

La requête "\^Xais\$" nous a permis de récupérer 81 lexèmes suffixés en -*ais* et dérivés de noms propres, la requête "\^Xois\$" 64 lexèmes suffixés en -*ois* et dérivés de noms propres. Pour -*ien*, nous avons obtenu 15565 lexèmes (dont 9017 sont restés après suppression des doublons). Quant à -*éen*, nous en avons récupéré 3194 (dont 1598 après suppression des doublons). Les résultats de ces quatre requêtes ont été exportés au format csv. Le bouton "Colonnes à exporter" permet de choisir les colonnes que l'on souhaite exporter et le format désiré (à noter que l'exportation n'opère que sur les résultats affichés à l'écran ; pour exporter l'ensemble des résultats d'une requête il faut donc penser à modifier le nombre de résultats de façon à ce que tous les résultats soient affichés sur une seule page, ou bien exporter chaque nouvelle page). 

## Présentation du jeu de données

Comme cela a été précisé ci-dessus, les données extraites de Demonette ne correspondent pas nécessairement à des gentilés dérivés de toponymes, mais sont tous des noms ou adjectifs dérivés de noms propres. En effet, l'état actuel de la ressource Démonette ne permet pas pour le moment d'obtenir directement les données souhaitées grâce un critère sémantique par exemple. Aussi, afin de garantir que les noms propres sont bien des toponymes, nous avons filtré les données extraites avec la base de données Prolex (Tran & Maurel, 2006) qui constitue une base de données de noms propres. Même ainsi, il reste du bruit, comme le cas de *bismarckien*, qui dérive bien d'un toponyme (*Bismarck*), mais qui correspond à un nom d'idéologie et non un gentilé. Une étape de validation manuelle des résultats est donc toujours nécessaire. 


Pour la partie prise en main de R qui suit, nous travaillons à partir de la table de données 'gentiles-tuto-demonext', qui comprend donc un sous-ensemble des données extraites de Démonette selon la méthodologie précédemment présentée. Ces données ont été annotées manuellement et semi-automatiquement pour différentes propriétés linguistiques. Nous ne présentons pas dans ce tutoriel les principes de l'annotation réalisée, cette dernière relevant des choix de chacun et chacune, mais nous en présentons les résultats dans le fichier qui accompagne le présent tutoriel. La table de données correspondante se présente sous la forme d'un fichier au format csv. Chaque ligne correspond à une paire toponyme-gentilé, et chaque colonne correspond à des informations relatives au toponyme et au dérivé.  
- Col. 1 : toponyme = la base du gentilé 
- Col. 2 : derive = le gentilé
- Col. 3 : transciption\_base = la transcription API de la base, obtenue à partir du Wiktionnaire (ressource GLAWI, Hathout & Sajous 2015)
- Col. 4 : transciption\_derive = la transcription API du dérivé, obtenue à partir du Wiktionnaire (ressource GLAWI, Hathout & Sajous 2015)
- Col. 5 : sfx\_gentile = le suffixe utilisé pour construire le gentilé
- Col. 6 : allomorphie = la base utilisée pour construire le gentilé est allormorphique (1) ou non (0)
- Col. 7 : topo\_compose = le toponyme est polylexical (1) ou non (0)
- Col. 8 : type\_topo = le toponyme renvoie à une ville, un pays, une région ou un département
- Col. 9 : pays = le toponyme renvoie à un lieu de France ou de l'étranger
- Col. 10 : nb\_syll\_base = le nombre de syllabes de la base
- Col. 11 : nb\_seg\_base = le nombre de phonèmes de la base
- Col. 12 : der\_seg\_base = le phonème final de la base



# Charger la table de données dans R

Dans ce tutoriel, nous utilisons des outils de base pour explorer la table de données, mais nous ne faisons pas une introduction à R. Si vous voulez plus de détails sur la façon dont les choses sont codées et sur la syntaxe R, nous vous renvoyons à un très bon tutoriel, qui, en plus d'expliquer les choses passées sous silence ici, présente différents outils pour explorer les données : https://juba.github.io/tidyverse/index.html


Pour charger la table de données, on utilise la commande `read.csv(x, sep=y)`, en spécifiant le chemin jusqu'au fichier (x), et le séparateur utilisé dans le fichier csv (y, ici la virgule). Il faut bien penser à changer le chemin du fichier afin de l'adapter à votre propre situation.

In [ ]:
gentiles <- read.csv("/path/to/file/gentiles-tuto-demonext.csv", sep=",")

Une autre façon de faire constiste à définir le répertoire où sont stockées les données comme répertoire de travail, grâce à la commande setwd(). Pour charger les données, on utilisera ensuite la même commande sans avoir à spécifier le chemin. 

In [ ]:
setwd("/path/to/file/")
gentiles <- read.csv("gentiles-tuto-demonext.csv", sep=",")

La table de données que nous venons de charger est un objet de type data frame. Pour s'en assurer on peut utiliser la commande `is.data.frame()`.

In [ ]:
is.data.frame(gentiles)

# Que contient la table de données ? 

On peut voir à quoi ressemble la table de données en regardant les premières lignes à l'aide de la commande `head()`, ou les dernières lignes à l'aide de la commande `tail()`. 

In [ ]:
head(gentiles)
tail(gentiles)

La commande `dim()` permet de connaitre les dimensions de la table de données : nombre de lignes et nombre de colonnes. La commande `summary()` permet d'avoir un résumé de chaque colonne : pour les variables numériques, on obtient les statistiques descriptives de base (moyenne, quartiles, médiane...) (cf. 'nb_syll_base'). Pour les variables catégorielles (cf. variable sfx_gentile), le résultat que l'on obtient varie en fonction de l'encodage de la variable. Si celle-ci est considérée comme un facteur, on obtient la population par catégorie (pour les 6 premières catégories s'il y en a plus). Si au contraire, elle est considérée comme une chaîne de caractères, seuls le nombre d'items (Length) et la nature de la variable (*character*) sont indiqués. Le passage d'un encodage à l'autre se fait à l'aide des commandes `as.factor()` et `as.character()` (comme illustré plus loin dans le tutoriel).

In [ ]:
dim(gentiles)
summary(gentiles)

Pour accéder au contenu d'une colonne, on utilise le \\$ après le nom de la table de données. S'affichent alors toutes valeurs de la colonne en partant de la première ligne et en allant jusqu'à la dernière.  

In [ ]:
gentiles$toponyme

Pour avoir un décompte de chaque catégorie dans une colonne donnée, on utilise la commande `table()` associée à une colonne. 

In [ ]:
table(gentiles$sfx_gentile)

Le tableau obtenu peut être enregistré sous la forme d'une nouvelle variable, en associant un nom de variable à la table en question, avec le signe <- (ou =).

In [ ]:
nbre_sfx <- table(gentiles$sfx_gentile)
nbre_sfx

De la même façon que pour le dataframe *gentiles*, on peut vérifier que l'objet *nbre_sfx* que l'on vient de créer est bien un tableau à l'aide de la commande `is.table()`.

In [ ]:
is.table(nbre_sfx)

Pour obtenir les proportions correspondantes aux effectifs de chaque catégorie, on utilisera la commande `prop.table()`.

In [ ]:
prop.table(nbre_sfx)

On peut éventuellement la multiplier par 100 pour avoir des pourcentages. 

In [ ]:
prop.table(nbre_sfx)*100

La commande `table()` permet également de faire des tableaux croisés, appelés tables de contingence. Elle nous permet, par exemple, de croiser les variables 'sfx_gentile' et 'allomorphie' afin de savoir, pour chaque suffixe, combien de gentilés présentent une allomorphie ou non (rappel pour la variable 'allomorphie', 1 = radical différent de la base ; 0 = aucune différence entre le radical et la base). 

In [ ]:
nbre_sfx_all <- table(gentiles$sfx_gentile, gentiles$allomorphie)
nbre_sfx_all

On peut regarder le pourcentage d'allomorphie pour chaque suffixe, en indiquant que les proportions doivent être faites par ligne (ajout de la valeur 1 dans la commande `prop.table()`). On peut aussi regarder comment se répartissent les suffixes dans les cas où il y a allomorphie et dans les cas où il n'y a pas allomorphie, en spécifiant que les proportions doivent être faites par colonne (ajout de la valeur 2). 

In [ ]:
prop.table(nbre_sfx_all, 1)*100
prop.table(nbre_sfx_all, 2)*100

Pour atteindre une case particulière de la table de données, nous utilisons les crochets `gentiles[x,y]`, où x renvoie à la ligne et y à la colonne. On peut par exemple donner les coordonnées : 3ème ligne et 4ème colonne. 

In [ ]:
gentiles[3,4]

On peut afficher le contenu d'une ligne entière en ne spécifiant pas la colonne.

In [ ]:
gentiles[3,]

Inversement, on peut afficher le contenu d'une ou plusieurs colonnes en ne spécifiant pas la ligne. Dans l'exemple suivant, on affiche les colonnes 1, 2, 3 et 4 en utilisant la syntaxe 1:4 qui permet de créer une suite de nombres entiers.

In [ ]:
gentiles[,1:4]

Cherchons maintenant à voir une ou des lignes contenant une information spécifique. Par exemple, nous souhaitons connaître toutes les informations pour la ville de Liège. Cela revient à demander la valeur particulière 'Liège' pour la colonne 'toponyme'. Pour cela, on spécifie le nom de la colonne et la valeur recherchée entre crochets. Dans l'exemple présenté ici, on choisit d'afficher toutes les colonnes des lignes qui contiennent la valeur qui nous intéresse (rien après la virgule). 

In [ ]:
gentiles[gentiles$toponyme=="Liège",]

De la même façon, on peut obtenir toutes les lignes contenant le suffixe -*éen*. Cette fois-ci on décide d'afficher uniquement les 6 premières colonnnes (valeur 1:6 après la virgule).  

In [ ]:
gentiles[gentiles$sfx_gentile=="éen", 1:6]

# Manipuler et créer des variables

Pour créer une nouvelle colonne dans la table de données, il suffit de définir un nom pour cette nouvelle colonne et de lui attribuer une ou des valeurs. Par exemple, on peut vouloir créer une variable qui donne le suffixe, sans distinguer les cas où -*ien* et -*éen* sont utilisés après les voyelles [i] et [e]. En d'autres termes, on veut fusionner les valeurs *ien* et *(i)en* d'un côté, et les valeurs *éen* et *(é)en* de l'autre. Pour cela, commençons par créer une variable dans notre table de données, en lui attribuant un contenu du type chaine de caractère, et en donnant la même valeur ('x') à chaque ligne.

In [ ]:
gentiles$suffixe <- "x"
table(gentiles$suffixe)

On peut également donner comme valeur à la nouvelle colonne les valeurs d'une autre colonne, dans notre exemple gentiles$sfx_gentile. 

In [ ]:
gentiles$suffixe <- gentiles$sfx_gentile
table(gentiles$suffixe)

Spécifions maintenant que dans les cas où la valeur est *(i)en*, il faut la remplacer par *ien* ; et dans les cas où la valeur est *(é)en*, il faut la remplacer par *éen*. Il y a plusieurs façon de faire. Une solution est d'utiliser les crochets, en indiquant que pour toutes les lignes où sfx_gentile est "(é)en" (resp. "(i)en"), la valeur de la colonne suffixe sera "éen" (resp. "ien"). 

In [ ]:
gentiles[gentiles$sfx_gentile=="(é)en", ]$suffixe <- "éen"
gentiles[gentiles$sfx_gentile=="(i)en", ]$suffixe <- "ien"
table(gentiles$suffixe)

**Remarque :** si la variable 'suffixe' est encodée comme facteur (noté *\<fct\>* dans les en-têtes de tableau) et non comme chaîne de caractères (noté *\<chr\>* dans les en-têtes de tableau), la table vous indiquera aussi les effectifs pour les valeurs '(é)en' et '(i)en', qui de fait seront de 0. Pour ne pas biaiser la suite des traitements, il faut donc supprimer ces deux valeurs. Une façon de faire est de transformer l'encodage de cette variable pour qu'elle soit considérée comme une chaîne de caractères. Cela se fait à l'aide de la commande `as.character()`. Les valeurs non représentées seront alors supprimées.

In [ ]:
gentiles$suffixe <- as.character(gentiles$suffixe)
table(gentiles$suffixe)
prop.table(table(gentiles$suffixe))*100

Si l'on veut maintenir l'encodage de la variable comme facteur, il suffit d'utiliser la commande `droplevels(x)`, qui permet de supprimer les valeurs non utilisées.

In [ ]:
gentiles$suffixe <- droplevels(gentiles$suffixe)
table(gentiles$suffixe)
prop.table(table(gentiles$suffixe))*100

On peut sauvegarder la table de données ainsi obtenue sous la forme d'un fichier csv grâce à la commande `write.csv(x,y)`, où x correspond à la table à sauvegarder, et y au fichier que l'on veut créer. Pensez à adapter le chemin du fichier en conséquence.

In [ ]:
gentiles <- write.csv(gentiles, "/path/to/file/gentiles.csv")

La table ainsi sauvegardée peut être à tout moment rechargée à l'aide de la commande `read.csv()`.

In [ ]:
gentiles <- read.csv("gentiles.csv", sep=",")

# Premières observations

Nous nous intéressons à l'effet des différentes variables dont nous disposons sur le choix entre les 4 suffixes quand un gentilé est créé à partir d'un toponyme. Pour cela, nous allons utiliser deux nouveaux packages : des ensembles d'outils spécifiques utiles à la visualisation des données. Il faut d'abord installer les packages avec `install.packages()`. Attention, l'installation initiale peut prendre un peu de temps.

In [ ]:
install.packages("questionr")

 Cette commande ne s'utilise qu'une seule fois, ensuite les packages sont installés dans le logiciel R et n'ont plus qu'à être chargés avec la fonction `library()`.

In [ ]:
library(questionr, tidyverse)

Regardons premièrement l'effet de la longueur de la base sur le choix du suffixe. La question est la suivante : la longueur de la base a-t-elle un impact sur le choix du suffixe ? Pour cela, nous croisons les variables "suffixe" et le nombre de syllabes de la base, dans une table de contingence, grâce à la commande `table()`.

In [ ]:
suff_long <-table(gentiles$nb_syll_base, gentiles$suffixe)
suff_long

Puis nous faisons une table des proportions des suffixes selon la longueur en utilisant la commande `prop.table()` (le paramètre 1 indique que nous voulons les proportions par ligne, nous multiplions le tout par 100 pour obtenir des pourcentages).

In [ ]:
prop.table(suff_long , 1)*100

Les bases de 5 et 6 syllabes sont très peu fréquentes : leur répartition selon les suffixes n'est donc pas très pertinente. Comme les bases de 4 syllabes sont également peu nombreuses par rapport aux bases plus courtes, nous regroupons les bases de 4 syllabes et plus (dont nous disons par convention qu'elles comportent 4 syllabes).

In [ ]:
gentiles$long_base <- 4
gentiles[gentiles$nb_syll_base=="1",]$long_base <- 1
gentiles[gentiles$nb_syll_base=="2",]$long_base <- 2
gentiles[gentiles$nb_syll_base=="3",]$long_base <- 3
suff_long_base <-table(gentiles$long_base, gentiles$suffixe)
suff_long_base
prop.table(suff_long_base, 1)*100

Les pourcentages montrent que les bases d'une syllabe favorisent légèrement les suffixes -*ois* et -*ais*. D'autre part, on constate que plus la base est longue, plus le suffixe -*ien* est  privilégié. 

Pour observer le nombre de phonèmes (plutôt que le nombre de syllabes), nous pouvons utiliser une boite à moustache : un graphique qui donne la distribution d'une variable numérique autour de la moyenne pour chaque suffixe. En premier lieu, il faut s'assurer que la variable 'nb\_seg\_base' est une variable numérique (à l'aide de la commande `is.numeric()`) et il faut transformer la variable suffixe en facteur (si ce n'est pas déjà le cas, avec la commande `as. factor()`). Ensuite, nous utilisons la fonction `boxplot()` en demandant que la variable 'suffixe' soit représentée sur l'axe des abscisses et  la variable 'nb\_seg\_base' sur l'axe des ordonnées, grâce à la formule "nb_seg_base \~  suffixe". 


In [ ]:
is.numeric(gentiles$nb_seg_base)
gentiles$suffixe <- as.factor(gentiles$suffixe)

boxplot(nb_seg_base ~  suffixe, data=gentiles)

Dans le graphique, la ligne en gras à l'intérieur des boîtes représente la médiane du nombre de phonèmes de la base pour chaque suffixe ; la boîte grise représente 50\% des observations (du 1er quartile au 3ème quartile). Les points sont considérés comme des valeurs extrêmes. Il n'y a pas de grande différence entre les distributions, hormis le fait que le suffixe -*ais* n'a pas de base de 2 phonèmes, contrairement aux autres suffixes, et que le suffixe -*éen* présente une médiane légèrement inférieure à celles des autres suffixes. 

Nous pouvons également obtenir les valeurs exactes correspondant aux données représentées dans ce graphique. Pour cela nous utilisons la fonction `summary()` qui donne les principaux éléments descriptifs concernant une variable quantitative.  La variable 'nb\_seg\_base' a donc une moyenne de 5,239 phonèmes et une médiane de 5 phonèmes. 

In [ ]:
summary(gentiles$nb_seg_base)

La fonction `tapply()` permet d'appliquer une fonction aux différentes catégories d'une variable plutôt qu'à la variable dans son ensemble. Il suffit de spécifier quel facteur doit être utilisé pour diviser les données qui nous intéressent. La commande `tapply()` prend donc trois arguments x, y et z : x désigne la variable sur laquelle on veut appliquer la commande z, et y le critère de division. Dans notre cas, nous souhaitons avoir les statistiques descriptives (z = `summary()`) de la variable  'nb\_seg\_base' (x), en fonction de chaque catégorie de suffixe (y = 'suffixe'). 

In [ ]:
tapply(gentiles$nb_seg_base, gentiles$suffixe, summary)

La moyenne du nombre de phonèmes de la base la plus basse est celle des gentilés en -*éen* ; la plus haute est celle des gentilés en -*ien*, ce qui va dans le sens de nos observations concernant le nombre de syllabes. 

Tournons-nous maintenant vers le phonème final de la base ('variable gentiles$der_seg_base'). Commençons par regarder la répartition des suffixes en fonction du dernier phonème. Etant donné que certains phonèmes se rencontrent rarement en fin de thème, il ne fait pas grand sens de regarder les proportions en l'état. 

In [ ]:
table(gentiles$der_seg_base, gentiles$suffixe)

En revanche, on peut regrouper les phonèmes finaux selon des caractéristiques phonologiques et regarder si ces traits ont une influence sur le choix du suffixe. Nous regroupons d'abord les phonèmes finaux en voyelles et consonnes. Nous créons une nouvelle variable ('gentiles\\$der\_seg\_voy\_cons') à laquelle nous donnons la valeur "cons", puis nous indiquons que les gentiles pour lesquels la valeur de la variable 'gentiles\\$der\_seg\_base' se trouve dans la liste des voyelles reçoivent la valeur "voy" pour la variable 'gentiles\\$der_seg_voy_cons'. Cela est obtenu à l'aide de l'instruction `%in%c()` qui nous permet de tester la variable 'der\_seg\_base' au regard d'une liste de voyelles, et ainsi de ne pas tester voyelle par voyelle.

Notez que nous avons choisi de classer les semi-consonnes dans la catégorie "cons".

In [ ]:
gentiles$der_seg_voy_cons <- "cons"
gentiles[gentiles$der_seg_base%in%c('i','e','ɛ','ɛ̃','a','y','ø','œ','u','o','ɔ','ɑ','ã','ə','ɔ̃','œ̃'),]$der_seg_voy_cons <- "voy"
table(gentiles$der_seg_voy_cons, gentiles$suffixe)
prop.table(table(gentiles$der_seg_voy_cons, gentiles$suffixe),1)*100

Les résultats montrent que le suffixe -*éen* est favorisé par une finale vocalique de la base, et que -*ais* et -*ois* semblent préférer les bases à finale consonantique. 

Nous créons maintenant une variable dans laquelle le dernier phonème est classé selon le mode d'articulation des consonnes et le degré d'aperture des voyelles.  Nous nommons cette nouvelle variable 'gentiles\\$der\_seg\_mode'. 

In [ ]:
gentiles$der_seg_mode <- ""
gentiles[gentiles$der_seg_base%in%c('v','f','z','s','ʒ','ʃ'),]$der_seg_mode <- "cons_fric"
gentiles[gentiles$der_seg_base%in%c('b','p','d','t','g','k'),]$der_seg_mode <- "cons_occl"
gentiles[gentiles$der_seg_base%in%c('m','n', 'ɲ' ),]$der_seg_mode <- "cons_nas"
gentiles[gentiles$der_seg_base%in%c('l','ʁ','j','ɥ','w'),]$der_seg_mode <- "cons_liq"
gentiles[gentiles$der_seg_base%in%c('i','y','u'),]$der_seg_mode <- "voy_ferm"
gentiles[gentiles$der_seg_base%in%c('e','ø','o'),]$der_seg_mode <- "voy_mi_f"
gentiles[gentiles$der_seg_base%in%c('ɛ','œ','ɔ','ə','œ̃',"ɛ̃","ɔ̃" ),]$der_seg_mode <- "voy_mi_o"
gentiles[gentiles$der_seg_base%in%c('a','ɑ','ã'),]$der_seg_mode <- "voy_ouv"
table(gentiles$der_seg_mode)
table(gentiles$der_seg_mode, gentiles$suffixe)
prop.table(table(gentiles$der_seg_mode, gentiles$suffixe),1)*100

Les consonnes fricatives sont très majoritairement suivies de -*ien*. Les consonnes nasales et occlusives sont particulièrement représentées avant les suffixes -*ais* et -*ois*. Les voyelles fermées se trouvent majoritairement avant le suffixe -*ien* et les mi-fermées avant -*éen*. Enfin, les voyelles ouvertes et mi-ouvertes semblent favoriser -*ien*, mais le nombre d'occurrences est moins élevé que pour les autres catégories.  

Nous créons maintenant une nouvelle variable pour prendre en compte le lieu d'articulation du phonème final de la base. 

In [ ]:
gentiles$der_seg_lieu <- ""
gentiles[gentiles$der_seg_base%in%c('ʁ','k','g','w'),]$der_seg_lieu <- "cons_arriere"
gentiles[gentiles$der_seg_base%in%c('b','p', 'm','f','v'),]$der_seg_lieu <- "cons_lab"
gentiles[gentiles$der_seg_base%in%c('d','t','z','s','n','l'),]$der_seg_lieu <- "cons_alv"
gentiles[gentiles$der_seg_base%in%c('ʒ','ʃ','ɲ','j','ɥ'),]$der_seg_lieu <- "cons_postalv"
gentiles[gentiles$der_seg_base%in%c( "ã", "o", "u", "ɔ","ɔ̃", 'ə'), ]$der_seg_lieu <- "voy_post"
gentiles[gentiles$der_seg_base%in%c('i','e','ɛ','ɛ̃','a','y','ø','œ', 'œ̃'  ), ]$der_seg_lieu <- "voy_ant"
table(gentiles$der_seg_lieu)
table(gentiles$der_seg_lieu, gentiles$suffixe)
prop.table(table(gentiles$der_seg_lieu, gentiles$suffixe),1)*100

Les consonnes labiales et postalvéolaires sont peu nombreuses et ne permettent donc pas de dégager des tendances, même si elles semblent globalement défavoriser -*ien*. Les consonnes arrières semblent favoriser les suffixes -*ois* et -*ien* ; les consonnes alvéolaires les suffixes -*ais* et -*ois*. Ces deux suffixes sont défavorisés par les voyelles antérieures, tandis que les voyelles postérieures attirent le suffixe -*ois* (ce sont en particulier les voyelles nasales postérieures /ɔ̃,ã/).  

Nous croisons maintenant le mode et le lieu d'articulation, pour observer si certains suffixes sont avantagés par l'interaction entre ces deux dimensions. 

In [ ]:
table(gentiles$der_seg_mode, gentiles$der_seg_lieu, gentiles$suffixe)

Si l'on regarde les pourcentages par suffixe, on peut observer que le croisement de certains traits phonologiques est favorable à un suffixe en particulier. Par exemple, la majorité des contextes où l'on rencontre le suffixe -*ais* est celui des consonnes alvéolaires (nous l'avions constaté précédemment), mais ce sont surtout les alvéolaires nasales (à savoir le phonème /n/) et occlusives (à savoir les phonèmes /t,d/) qui sont représentées. Pour le suffixe -*ien*, on observe que le contexte présentant le plus de gentilés avec ce suffixe est une voyelle antérieure fermée. Le suffixe -*éen*, quant à lui, se rencontre très majoritairement après une voyelle, et dans quasiment la moitié des cas après une voyelle mi-fermée. Enfin, les contextes d'apparition de -*ois* montrent que le contexte consonnantique, en particulier avec des consonnes arrières ou alvéolaires, semble être un contexte particulièrement représenté. 

Notons ici que ce résultat est obtenu en indiquant dans la commande `prop.table()` l'argument `3` et non l'argument `1` ou `2` mis jusque-là. Ces derniers nous auraient par exemple donné respectivement le pourcentage de consonnes fricatives en fonction du lieu d'articulation, ou le pourcentage de consonnes alvéolaires en fonction du mode d'articulation *sur l'ensemble des quatre suffixes*. L'argument `3` nous permet ici d'avoir le pourcentage d'un croisement de traits donné *par suffixe* ; il indique par exemple que 12\% des gentilés en -*ais* impliquent une consonne fricative alvéoaire, les 88\% de gentilés en -*ais* restant n'impliquant pas ce croisement donné de traits.

In [ ]:
 prop.table(table(gentiles$der_seg_mode, gentiles$der_seg_lieu,gentiles$suffixe),3)*100

La variable 'topo\_compose' permet de savoir si le toponyme sur lequel est formé le gentilé est polylexical (1) comme dans Villeloin-Coulangé, ou non (0). La variable 'allomorphie' spécifie si la base du gentilé connaît une forme d'allomorphie (1) ou non (0). Dans l'optique d'observer si ces deux dimensions peuvent jouer un rôle sur la distribution des suffixes, nous croisons chaque variable avec la variable 'suffixe', et nous donnons les pourcentages. 

In [ ]:
sfx_topo_comp <- table(gentiles$topo_compose, gentiles$suffixe)
sfx_topo_comp
prop.table(sfx_topo_comp, 1)*100

In [ ]:
sfx_allom <- table(gentiles$allomorphie, gentiles$suffixe)
sfx_allom
prop.table(sfx_allom, 1)*100

Dans les données que nous étudions, les suffixes -*ois* et -*ais* se rencontrent très rarement avec un toponyme composé de plusieurs unités. L'allomorphie de la base semble être plus présente avec le suffixe -*ien* qu'avec les autres. 

Concernant les aspects extra-linguistiques, nous regardons si le choix des suffixes est corrélé au fait que les toponymes soient français ou étrangers. 

In [ ]:
sfx_pays<- table(gentiles$pays, gentiles$suffixe)
sfx_pays
prop.table(sfx_pays,1)*100

Les suffixes -*ois* et -*ais* semblent être légèrement favorisés avec des toponymes désignant des lieux hors de  France, alors que -*ien* et -*éen* seraient plutôt préférés pour les toponymes français.

Nous nous intéressons maintenant à la variable qui indique ce que dénote le toponyme (type\_topo).

In [ ]:
table(gentiles$type_topo)

Les villes sont très largement représentées, et dans une moindre mesure les pays et les régions. La catégorie "autre" comprend des cas marginaux, que l'on peut consulter avec la commande suivante :

In [ ]:
gentiles[gentiles$type_topo=="autre", c(1:6)]

Afin de voir si le type de toponyme a un effet sur le choix du suffixe, nous croisons cette variable avec la variable 'suffixe'. 

In [ ]:
sfx_type_topo <- table(gentiles$type_topo, gentiles$suffixe)
sfx_type_topo
prop.table(sfx_type_topo, 1)*100

Nous observons que les bases désignant un pays favorisent le suffixe -*ais*, tandis que celles dénotant une ville semblent être plus souvent suffixées en -*éen*. En ce qui concerne les régions et les départements, il semble que la distribution de la variable suffixe soit similaire, ce qui nous pousse à fusionner ces deux catégories. Nous créons ensuite une nouvelle table de contingence, ainsi que la table contenant les pourcentages, prenons en compte cette fusion. 

In [ ]:
gentiles$type_topo2<- gentiles$type_topo
gentiles[gentiles$type_topo%in%c("région", "département"), ]$type_topo2 <- "reg_dep"
sfx_type_topo <- table(gentiles$type_topo2, gentiles$suffixe)
sfx_type_topo
prop.table(sfx_type_topo, 1)*100

Nous enregistrons cette nouvelle version de la table de données, contenant les variables concernant la longueur de la base et la phonologie du segment final de la base. 

Attention, pensez à donner un nom de fichier différent de celui créé précédemment si vous ne souhaitez pas écraser la version sauvegardée plus tôt. Sinon, votre nouveau fichier sera réécrit sur le précédent, effacant l'état antérieur de votre table.

In [ ]:
gentiles <- write.csv(gentiles, "/path/to/file/gentiles_v2.csv")

# Tests statistiques

Les tableaux précédents donnent des tendances, mais ne donnent aucune indication sur le fait que ces tendances sont le reflet d'une véritable corrélation entre les variables ou sur le fait qu'au contraire, ces distributions n'ont rien d'étonnant étant donné le hasard de l'échantillonage. Autrement dit, les tendances observées ne disent rien quant à la significativté statistique des différentes distributions observées. Pour statuer sur la significativité, il faut utiliser des tests statistiques. Ce tutoriel n'a pas vocation à délivrer une introduction aux statistiques, ainsi pour plus de détails sur les tests statistiques, reportez-vous à d'autres ressources telles que : 
- https://www.institutpasteur.nc/wp-content/uploads/2017/09/20141107_BouquinRComplet.pdf (en français, p. 111 pour les tests statistiques)
- http://people.linguistics.mcgill.ca/~morgan/qmld-book/hypothesis-testing.html (en anglais, sur des données linguistiques)

Nous illustrons l'utilisation du test du chi-deux (ou chi-square) pour les variables 'suffixe' et 'pays' : la distribution des suffixes est-elle dépendante de la variable pays ? 
Nous créons un dataframe qui reprend le contenu de la table de contingence "sfx_pays" créée précédemment. Puis nous lui appliquons le test du  chi-square de Pearson à l'aide de la commande `chisq.test()`, puisque nous avons à faire à des variables discrètes.

In [ ]:
df_sfx_pays <- as.data.frame.matrix(sfx_pays)

chisq.test(df_sfx_pays)

Le résultat qui nous intéresse particulièrement ici est la valeur de la p-value. Comme elle est inférieure à 0,05, on peut conclure que la distribution s'éloigne de la distribution attendue si les deux variables étaient indépendantes, à un degré assez important pour qu'on puisse supposer que ce n'est pas le fruit du hasard. En d'autres termes, le résultat de ce test statistique nous permet de soutenir l'idée selon laquelle les deux variables (suffixe et pays) ne sont pas indépendantes (mais l'interprétation causale reste à la charge du chercheur ou de de la chercheuse...). 

Nous pouvons utiliser le même test statistique pour évaluer l'indépendance des variables 'suffixe' et 'type\_topo2'. Cependant, la variable 'type_topo2' présente une valeur "autre" qui ne représente que 3 toponymes et qu'il est nécessaire d'écarter pour obtenir une table de contingence dont la distribution est plus homogène.  Pour cela, nous utilisons la fonction `subset()` qui permet d'obtenir un sous-ensemble d'une table de données, en spécifiant le ou les critères de tri. 

In [ ]:
gentiles2 <- subset(gentiles, gentiles$type_topo2 != "autre")
head(gentiles2)
sfx_type_topo <- table(gentiles2$type_topo2, gentiles2$suffixe)
sfx_type_topo
df_sfx_type_topo <- as.data.frame.matrix(sfx_type_topo)
chisq.test(df_sfx_type_topo)

Comme dans l'exemple précédent, la valeur de la p-value est inférieure à 0,05 ce qui signifie que la distribution des suffixes selon le type de toponyme est suffisamment éloignée de la distribution attendue si les deux variables étaient indépendantes, pour pouvoir raisonnablement conclure qu'elles sont dépendantes. 

Pour plus de détail sur le test de chi-square, nous renvoyons vers des ouvrages de statistique. 
D'autres tests sont possibles en fonction du type de données évaluées : `wilcox.test()` pour le test de Wilcoxon (ou de Mann-Whitney), `t.test()` pour le test t de Student, `kruskal.test()` pour le test de Kruskal-Wallis...

